In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix

from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

In [9]:
# 1. Veri Yükleme
data = pd.read_csv('/content/Student performance in mathematics.csv')

In [10]:
# 2. Hedef Değişken Oluşturma
data['average_score'] = (data['Math score'] + data['Reading score'] + data['Writing score']) / 3

def assign_label(avg):
    if avg < 60:
        return 0  # Başarısız
    elif avg < 80:
        return 1  # Orta
    else:
        return 2  # Başarılı

data['label'] = data['average_score'].apply(assign_label)

In [11]:
# 3. Özellik ve hedef ayrımı
X = data[['Gender', 'Race/ethnicity', 'Parental level of education', 'Lunch', 'Test preparation course',
          'Math score', 'Reading score', 'Writing score']]
y = data['label']

In [12]:
# 4. Preprocessing
categorical_cols = ['Gender', 'Race/ethnicity', 'Parental level of education', 'Lunch', 'Test preparation course']
numeric_cols = ['Math score', 'Reading score', 'Writing score']

preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(drop='first'), categorical_cols),
    ('num', StandardScaler(), numeric_cols)
])

X_processed = preprocessor.fit_transform(X)

In [13]:
# 5. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.4, random_state=42, stratify=y)

print("Eğitim sınıf dağılımı:", pd.Series(y_train).value_counts(normalize=True))
print("Test sınıf dağılımı:", pd.Series(y_test).value_counts(normalize=True))

Eğitim sınıf dağılımı: label
1   0.49
0   0.26
2   0.25
Name: proportion, dtype: float64
Test sınıf dağılımı: label
1   0.48
0   0.27
2   0.25
Name: proportion, dtype: float64


In [19]:
# 6. SMOTE ile Sınıf Dengesi
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Veri Dağılımını Kontrol Et
print("SMOTE sonrası sınıf dağılımı:", pd.Series(y_train).value_counts())

SMOTE sonrası sınıf dağılımı: label
0    61
1    61
2    61
Name: count, dtype: int64


In [14]:
# 7. One-hot Encoding for y
y_train_categorical = to_categorical(y_train, num_classes=3)
y_test_categorical = to_categorical(y_test, num_classes=3)

Lazy Prediction

In [1]:
!pip install lazypredict

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.6 MB/s eta 0:00:00


In [15]:
# LazyPredict için gerekli import
from lazypredict.Supervised import LazyClassifier

In [16]:
# LazyPredict ile Klasik Modellerin Performansını Ölçelim
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

In [17]:
# LazyPredict Eğitimi
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000643 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 124
[LightGBM] [Info] Number of data points in the train set: 124, number of used features: 12
[LightGBM] [Info] Start training from score -1.354546
[LightGBM] [Info] Start training from score -0.709408
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

In [18]:
# Modellerin Performansları
print("\nLazyPredict Modellerinin Performansı:\n")
models


LazyPredict Modellerinin Performansı:



,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
AdaBoostClassifier,1.00,1.00,None,1.00,0.16
XGBClassifier,0.99,0.99,None,0.99,0.17
BaggingClassifier,0.99,0.98,None,0.99,0.06
DecisionTreeClassifier,0.99,0.98,None,0.99,0.02
LogisticRegression,0.99,0.98,None,0.99,0.03
RandomForestClassifier,0.99,0.98,None,0.99,0.22
ExtraTreesClassifier,0.98,0.98,None,0.98,0.17
LGBMClassifier,0.98,0.98,None,0.98,0.14
LinearDiscriminantAnalysis,0.96,0.97,None,0.96,0.05
